# Extract SMILES, IUPAC from Pubchem XML Files
## Zip of data on Google Drive

In [1]:
import os
import gzip
import xml.etree.ElementTree as ET

In [16]:
path = "/media/external/pubchem_compound_2d/XML/"
out_path = "/media/external/pubchem_compound_2d/SMILES_IUPAC_CSV/"
files = os.listdir(path)

In [5]:
print(len(files))
print(files[0])

5392
Compound_042475001_042500000.xml.gz


In [ ]:
for f in files:
    #read XML file
    unzipped = gzip.open(path + f, 'rb')
    lines = unzipped.read()
    splitlines = lines.decode().split("\n")
    unzipped.close()
    
    #open csv
    outfile = open(out_path + f, 'w')
    outfile.write("id,SMILES,IUPAC\n")
    
    records_counter, failed_counter = 0, 0
    
    start_line = 0
    while "<PC-Compound>" not in splitlines[start_line]:
        start_line += 1
        
    line = start_line
    while "</PC-Compounds>" not in splitlines[line]:
        start_line = line
        while "</PC-Compound>" not in splitlines[line]:
            line += 1
        compound_lines = "\n".join(splitlines[start_line:line+1])
        line += 1
        
        #build tree of XML tags
        child = ET.fromstring(compound_lines)
    
        #get pubchem CID (first record)
        id = int(child[0][0][0][0].text)

        #find which group has molecule properties (SMILES / IUPAC)
        props = ""
        for group in child:
            if "PC-Compound_props" in group.tag:
                props = group

        #Get SMILES, IUPAC
        smiles, iupac = "", ""
        for infodata in props:
            label = infodata[0][0][0]
            name = infodata[0][0][1]
            value = infodata[1]
            if label.text == "IUPAC Name" and name.text == "Allowed":
                iupac = value[0].text
            if label.text == "SMILES" and name.text == "Canonical":
                smiles = value[0].text

        if len(smiles) == 0 or len(iupac) == 0:
            failed_counter += 1
            #print("error reading, skipping", id)
            #print(smiles, iupac)
        else:
            records_counter += 1
            outfile.write(str(id) + "," + smiles + "," + iupac + "\n")

    outfile.close()
    print(f, records_counter, "added,", failed_counter, "skipped.")

Compound_042475001_042500000.xml.gz 7654 added, 168 skipped.
Compound_035725001_035750000.xml.gz 1841 added, 0 skipped.
Compound_069275001_069300000.xml.gz 24111 added, 254 skipped.
Compound_062900001_062925000.xml.gz 24996 added, 4 skipped.
Compound_009575001_009600000.xml.gz 24218 added, 331 skipped.
Compound_079175001_079200000.xml.gz 24995 added, 5 skipped.
Compound_020750001_020775000.xml.gz 19739 added, 840 skipped.
Compound_069150001_069175000.xml.gz 24006 added, 324 skipped.
Compound_056350001_056375000.xml.gz 24978 added, 22 skipped.
Compound_068100001_068125000.xml.gz 22752 added, 620 skipped.
Compound_122225001_122250000.xml.gz 23829 added, 1130 skipped.
Compound_102950001_102975000.xml.gz 24961 added, 39 skipped.
Compound_047225001_047250000.xml.gz 3967 added, 1 skipped.
Compound_021125001_021150000.xml.gz 21240 added, 933 skipped.
Compound_003725001_003750000.xml.gz 22748 added, 1050 skipped.
Compound_040100001_040125000.xml.gz 6573 added, 7 skipped.
Compound_094500001_094